In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO
from postgres_driver import PostgresDatabaseDriver

In [2]:
url = 'https://www.pro-football-reference.com/years/2024/fantasy.htm#fantasy'

In [3]:
response = requests.get(url)

In [4]:
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
table = soup.find('table')
table_html = str(table)
table_io = StringIO(table_html)

In [6]:
df = pd.read_html(table_io, header=None)[0]

In [7]:
df.columns = [None] * len(df.columns)
df.columns = ['player_rank', 'player_name', 'team', 'position', 'age', 'games_played', 'games_started', 'pass_comp', 'pass_att', 'pass_yds', 'pass_tds', 'pass_int', 'rush_att', 'rush_yds', 'rush_yds_per_att', 'rush_tds', 'rec_tgt', 'receptions', 'rec_yds', 'rec_yds_per_catch', 'rec_tds', 'fumbles', 'fumbles_lost', 'total_tds', 'two_pt_conv', 'two_pp', 'fantasy_points', 'fantasy_ppr', 'fantasy_dk', 'fantasy_fd', 'fantasy_vbd', 'fantasy_pos_rank', 'fantasy_ovr_rank']

In [8]:
driver = PostgresDatabaseDriver()

In [9]:
engine = driver.create_engine()

In [10]:
df.to_sql('player_stats', engine, if_exists='replace', index=False, schema='landing')

508

In [11]:
driver.close()